In [1]:
# !pip install albumentations==0.4.5

In [2]:

from datetime import datetime
print("Current Date/Time: ", datetime.now())


Current Date/Time:  2020-09-26 23:05:13.842341


In [3]:
%matplotlib inline
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import numpy as np
import albumentations as A
from albumentations.pytorch import ToTensor
from data.transformations import Transformations
from data.downloader import Downloader
from data.dataloader import Dataloader
from data.utils import cal_mean, cal_std, image_size
from tqdm import tqdm
from model.regularizer import l1_regularization
from model.train import train 
from model.test import val
from model.network import *
from model.QuizDNN import *
import matplotlib.pyplot as plt
from plotter import *
import cv2
from gradcam import *

In [4]:
downloader = Downloader('./data', 'CIFAR10')
sample = downloader.download_sample(train = True)

Files already downloaded and verified


In [5]:
args = {
    'mean': (0.5, 0.5, 0.5),
    'std': (0.5, 0.5, 0.5),
    'train': False
}
args['train'] = True
args['horizontal_flip_prob'] = 0.2
args['vertical_flip_prob'] = 0.0
args['gaussian_blur_prob'] = 0.0
args['rotate_degree'] = 20
args['cutout'] = 0.3
args['cutout_height'] = 16
args['cutout_width'] = 16
args['mean'] = cal_mean(sample)  # calculate mean
args['std'] = cal_std(sample)   # calculate standard deviation
# print(args['mean'])
classes = sample.classes
print(classes)

['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


In [6]:
transformations = Transformations(**args)
trainset = downloader.download(transformations)
args['train'] = False
transformations = Transformations(**args)
testset = downloader.download(transformations, train = False)

Files already downloaded and verified
Files already downloaded and verified


In [7]:
batch_size = 64
num_workers = 2
trainloader = Dataloader.getloader(trainset, batch_size, num_workers)
testloader = Dataloader.getloader(testset, batch_size, num_workers, train = False)

In [8]:
cuda = torch.cuda.is_available()
print('GPU Available?', cuda)
device = torch.device("cuda" if cuda else "cpu")
torch.manual_seed(1)
if cuda:
    torch.cuda.manual_seed(1)

GPU Available? True


In [9]:
from torchsummary import summary
model = Net().to(device)  # Create model
size = image_size(sample)
print(summary(model, input_size = size))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 32, 32]             896
              ReLU-2           [-1, 32, 32, 32]               0
       BatchNorm2d-3           [-1, 32, 32, 32]              64
            Conv2d-4           [-1, 32, 32, 32]           9,248
              ReLU-5           [-1, 32, 32, 32]               0
       BatchNorm2d-6           [-1, 32, 32, 32]              64
            Conv2d-7           [-1, 32, 32, 32]           9,248
              ReLU-8           [-1, 32, 32, 32]               0
       BatchNorm2d-9           [-1, 32, 32, 32]              64
        MaxPool2d-10           [-1, 32, 16, 16]               0
           Conv2d-11           [-1, 32, 16, 16]           9,248
             ReLU-12           [-1, 32, 16, 16]               0
      BatchNorm2d-13           [-1, 32, 16, 16]              64
           Conv2d-14           [-1, 32,

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=0.0)
StepLR(optimizer, step_size=25, gamma=0.1)

In [11]:
totalepochs = 15
losses, accuracies = [], []
for epoch in range(1, totalepochs + 1):
    print(f'Epoch {epoch}:')
    train(model, trainloader, device, optimizer, criterion)
    loss, accuracy = val(model, testloader, device, criterion)
    losses.append(loss)
    accuracies.append(accuracy)

  0%|          | 0/782 [00:00<?, ?it/s]

Epoch 1:


  0%|          | 0/782 [00:00<?, ?it/s]


Validation set: Average loss: 0.0175, Accuracy: 5978/10000 (59.78%)

Epoch 2:


  0%|          | 0/782 [00:00<?, ?it/s]

...


Validation set: Average loss: 0.0089, Accuracy: 8080/10000 (80.80%)

Epoch 14:


  0%|          | 0/782 [00:00<?, ?it/s]


Validation set: Average loss: 0.0087, Accuracy: 8090/10000 (80.90%)

Epoch 15:


Loss=0.66 Batch_ID=781 Accuracy=81.10: 100%|██████████| 782/782 [00:16<00:00, 46.31it/s]



Validation set: Average loss: 0.0093, Accuracy: 8022/10000 (80.22%)

